In [2]:
import gpxpy
import matplotlib.pyplot as plt
import datetime
from geopy import distance
from math import sqrt, floor
import numpy as np
import pandas as pd

### Read in gpx File

In [12]:
gpx_file = open('Ride_for_AIDS_Pt1.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

data = gpx.tracks[0].segments[0].points

### Create Data Frame from GPX File

In [14]:
df = pd.DataFrame(columns=['lon', 'lat', 'alt', 'time'])
for point in data:
    df = df.append({'lon': point.longitude, 'lat' : point.latitude, 'alt' : point.elevation, 'time' : point.time}, ignore_index=True)

# Set Timestampes
df['time'] = pd.DatetimeIndex([i.replace(tzinfo=None) for i in df['time']])
df.head()

### Read in Glucose Data

In [25]:
Day1 = pd.read_csv('data-9.csv',sep='\t')
Day2 = pd.read_csv('data-10.csv',sep='\t')
Day3 = pd.read_csv('data-11.csv',sep='\t')
DexcomData = pd.concat([Day1, Day2, Day3])
DexcomData.columns = ['time', 'glucose']
DexcomData['time'] = pd.to_datetime(DexcomData['time']) 
DexcomData.head()

,time,glucose
0,2017-07-08 01:37:00,245.0
1,2017-07-08 01:42:00,258.0
2,2017-07-08 01:47:00,268.0
3,2017-07-08 01:52:00,281.0
4,2017-07-08 01:57:00,290.0


### Merge Glucose Data with Strava Data

In [51]:
All_Data = pd.merge(df, DexcomData, on='time', how='left').fillna(method='ffill')
All_Data.head()

,lon,lat,alt,time,glucose
0,-87.685783,42.062315,186.9,2017-07-08 11:43:25,NaN
1,-87.685712,42.062217,186.9,2017-07-08 11:43:43,NaN
2,-87.685740,42.062183,186.9,2017-07-08 11:43:45,NaN
3,-87.685783,42.062169,186.8,2017-07-08 11:43:47,NaN
4,-87.685797,42.062133,186.8,2017-07-08 11:43:49,NaN


In [52]:
All_Data.to_csv('Strava_Glucose.csv',index=False,sep=',')